In [20]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Helpers
import sys
from datetime import datetime
import pickle

from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,log_loss
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut,KFold,StratifiedKFold
from sklearn.decomposition import PCA,KernelPCA
import lightgbm as lgb

import sys
sys.path.insert(0,'../../') 
from helpers import *
from MAP_estimator import MAP_estimator

# Algorithm
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'

In [21]:
count_df = pd.read_pickle(data_path+'count_df.pkl')
bin_df = pd.read_pickle(data_path+'bin_df.pkl')
freq_df = pd.read_pickle(data_path+'freq_df.pkl')
train_y = pd.read_csv(data_path+'train_y.csv')

In [22]:
count_df.head()

,sample_name,CAISEDSSYNSPLHF,CAISESTGNTEAFF,CAISESVSGGNNEQFF,CAISGDSYNEQFF,CAISQGMNTEAFF,CASGGNTIYF,CASGGSSYEQYF,CASGGVEAFF,CASGLSGTEAFF,...,CASSQEGGYGYTF,CASSSGTNNEQFF,CASSVDNTGELFF,CASSVVSGGTEAFF,CATSRLAGGYNEQFF,CSARDLAASSYNEQFF,CSARDLSSSYEQYF,CSARDRPYTGELFF,CSARGLAGGPNEQFF,CSARTGLGNTIYF
,,TCRBV10,TCRBV10,TCRBV10,TCRBV10,TCRBV10,TCRBV02,TCRBV27,TCRBV02,TCRBV12,...,TCRBV04,TCRBV12,TCRBV06,TCRBV09,TCRBV15,TCRBV20,TCRBV20,TCRBV20,TCRBV20,TCRBV20
,,TCRBV10-03,TCRBV10-03,TCRBV10-03,TCRBV10-03,TCRBV10-03,TCRBV02-01,TCRBV27-01,TCRBV02-01,TCRBV12-05,...,TCRBV04-02,null,TCRBV06-04,TCRBV09-01,TCRBV15-01,null,null,null,null,null
,,01,01,01,01,01,01,01,01,01,...,01,null,null,null,01,null,null,null,null,null
,,TCRBJ01,TCRBJ01,TCRBJ02,TCRBJ02,TCRBJ01,TCRBJ01,TCRBJ02,TCRBJ01,TCRBJ01,...,TCRBJ01,TCRBJ02,TCRBJ02,TCRBJ01,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ02,TCRBJ01
,,TCRBJ01-06,TCRBJ01-01,TCRBJ02-01,TCRBJ02-01,TCRBJ01-01,TCRBJ01-03,TCRBJ02-07,TCRBJ01-01,TCRBJ01-01,...,TCRBJ01-02,TCRBJ02-01,TCRBJ02-02,TCRBJ01-01,TCRBJ02-01,TCRBJ02-01,TCRBJ02-07,TCRBJ02-02,TCRBJ02-01,TCRBJ01-03
,,01,01,01,01,01,01,01,01,01,...,01,01,01,01,01,01,01,01,01,01
0,RA47,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,HC9,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RA29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,RA8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
count_df = count_df.merge(train_y,on='sample_name')
bin_df = bin_df.merge(train_y,on='sample_name')
freq_df = freq_df.merge(train_y,on='sample_name')

In [25]:
c = count_df.columns.values.tolist()[1]
print(c)

('sample_name', '', '', '', '', '', '')


In [26]:
count_df.drop(c,inplace=True,axis=1)

In [27]:
TCRs = count_df.drop(['sample_name'],axis=1).columns.values
len(TCRs)

33769

In [51]:
clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                        learning_rate=0.1, max_depth=-1, min_child_samples=30,min_data=1,
                        min_child_weight=0.001, min_split_gain=0.0, n_estimators=200,
                        n_jobs=-1, num_leaves=28, objective='binary', random_state=0,
                        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                        subsample_for_bin=200000, subsample_freq=1)

In [67]:
clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                        learning_rate=0.1, max_depth=-1, min_child_samples=30,min_data=1,
                        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
                        n_jobs=-1, num_leaves=32, objective='binary', random_state=0,
                        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                        subsample_for_bin=200000, subsample_freq=1)
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

kf = LeaveOneOut()
for train_index,test_index in kf.split(count_df):
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    print(test_cv['sample_name'].tolist()[0])
    
    asso_TCRs = TCRs
    
    clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
    test_pred = clf.predict(test_cv[asso_TCRs])[0]
    test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1][0]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_true.append(test_cv['phenotype_status'].values[0])
    y_pred.append(test_pred)
    y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

print('cv auroc:',roc_auc_score(y_true,y_proba))
print('cv log_loss:',log_loss(y_true,y_proba))

RA47
y_true: 1 y_pred: 1 posterior_c1: 0.9947474443686565

HC9
y_true: 0 y_pred: 1 posterior_c1: 0.9996947481367998

RA29
y_true: 1 y_pred: 0 posterior_c1: 0.00022478014263945122

RA8
y_true: 1 y_pred: 1 posterior_c1: 0.999984247189324

RA63
y_true: 1 y_pred: 1 posterior_c1: 0.9999812253945528

RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9988858841186115

HC17
y_true: 0 y_pred: 1 posterior_c1: 0.9961286220733776

HC3
y_true: 0 y_pred: 1 posterior_c1: 0.9999855889791969

RA17
y_true: 1 y_pred: 1 posterior_c1: 0.999981184378612

RA46
y_true: 1 y_pred: 1 posterior_c1: 0.9938664021416284

HC6
y_true: 0 y_pred: 1 posterior_c1: 0.989240937274781

RA26
y_true: 1 y_pred: 0 posterior_c1: 0.03885708671773383

RA15
y_true: 1 y_pred: 0 posterior_c1: 0.15688013979382437

RA53
y_true: 1 y_pred: 0 posterior_c1: 0.013534730722389648

RA32
y_true: 1 y_pred: 0 posterior_c1: 0.023827742016757504

RA40
y_true: 1 y_pred: 1 posterior_c1: 0.999944040565093

HC8
y_true: 0 y_pred: 1 posterior_c1: 0.99490408034367

In [63]:
prams = {'min_child_samples':[5,10,15]}
scoring = 'roc_auc'
outer = StratifiedKFold(n_splits=5)
inner = StratifiedKFold(n_splits=5)
def nestedCrossValidation(X, y, clf, param_dist, scoring, outer, inner):
    rs = GridSearchCV(clf, param_dist, verbose=1, scoring=scoring, cv=inner)
    rs.fit(X, y)
    scores = cross_val_score(rs, X, y, cv=outer)
    return ((rs.best_score_, rs.best_params_), scores)

In [59]:
train_X = count_df[TCRs].values
train_y = count_df['phenotype_status']

In [64]:
nestedCrossValidation(train_X,train_y,clf,prams,scoring,outer,inner)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.2min finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   59.4s finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   57.1s finished


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   52.3s finished


((0.576923076923077, {'min_child_samples': 5}),
 array([0.55769231, 0.55769231, 0.48076923, 0.69230769, 0.59615385]))